<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#TTD-notebook:-testing-code,-EDA" data-toc-modified-id="TTD-notebook:-testing-code,-EDA-1">TTD notebook: testing code, EDA</a></span><ul class="toc-item"><li><span><a href="#Parse-header---define-function!" data-toc-modified-id="Parse-header---define-function!-1.1">Parse header - define function!</a></span></li></ul></li><li><span><a href="#ID-mapping" data-toc-modified-id="ID-mapping-2">ID-mapping</a></span><ul class="toc-item"><li><span><a href="#P1-03:-TTD-Drug-IDs--&gt;-PUBCHEM.COMPOUND,-CAS,-CHEBI" data-toc-modified-id="P1-03:-TTD-Drug-IDs--&gt;-PUBCHEM.COMPOUND,-CAS,-CHEBI-2.1">P1-03: TTD Drug IDs -&gt; PUBCHEM.COMPOUND, CAS, CHEBI</a></span><ul class="toc-item"><li><span><a href="#Parse-header" data-toc-modified-id="Parse-header-2.1.1">Parse header</a></span></li><li><span><a href="#EDA-parse-mappings" data-toc-modified-id="EDA-parse-mappings-2.1.2">EDA parse mappings</a></span></li><li><span><a href="#Parser-code" data-toc-modified-id="Parser-code-2.1.3">Parser code</a></span></li></ul></li><li><span><a href="#P2-01:-TTD-Target-IDs--&gt;-UniProtKB-names--&gt;-IDs" data-toc-modified-id="P2-01:-TTD-Target-IDs--&gt;-UniProtKB-names--&gt;-IDs-2.2">P2-01: TTD Target IDs -&gt; UniProtKB names -&gt; IDs</a></span><ul class="toc-item"><li><span><a href="#Parse-header" data-toc-modified-id="Parse-header-2.2.1">Parse header</a></span></li><li><span><a href="#EDA-grab-content" data-toc-modified-id="EDA-grab-content-2.2.2">EDA grab content</a></span></li><li><span><a href="#Parser---grab-content" data-toc-modified-id="Parser---grab-content-2.2.3">Parser - grab content</a></span></li><li><span><a href="#Map-Uniprot-names--&gt;-IDs" data-toc-modified-id="Map-Uniprot-names--&gt;-IDs-2.2.4">Map Uniprot names -&gt; IDs</a></span></li><li><span><a href="#Finish-constructing-mappings" data-toc-modified-id="Finish-constructing-mappings-2.2.5">Finish constructing mappings</a></span></li></ul></li></ul></li><li><span><a href="#Edges" data-toc-modified-id="Edges-3">Edges</a></span><ul class="toc-item"><li><span><a href="#P1-05:-&quot;drug-treats-indication&quot;" data-toc-modified-id="P1-05:-&quot;drug-treats-indication&quot;-3.1">P1-05: "drug treats indication"</a></span><ul class="toc-item"><li><span><a href="#Parse-header" data-toc-modified-id="Parse-header-3.1.1">Parse header</a></span></li><li><span><a href="#EDA-grab-content" data-toc-modified-id="EDA-grab-content-3.1.2">EDA grab content</a></span></li><li><span><a href="#Parser---grab-content" data-toc-modified-id="Parser---grab-content-3.1.3">Parser - grab content</a></span></li><li><span><a href="#Map-clinical_status" data-toc-modified-id="Map-clinical_status-3.1.4">Map clinical_status</a></span></li><li><span><a href="#Map-TTD-drug-IDs" data-toc-modified-id="Map-TTD-drug-IDs-3.1.5">Map TTD drug IDs</a></span></li><li><span><a href="#Filter-out-some-indication-names" data-toc-modified-id="Filter-out-some-indication-names-3.1.6">Filter out some indication names</a></span></li><li><span><a href="#EDA-map-indication-names" data-toc-modified-id="EDA-map-indication-names-3.1.7">EDA map indication names</a></span></li><li><span><a href="#Parser-map-indication-names" data-toc-modified-id="Parser-map-indication-names-3.1.8">Parser map indication names</a></span></li><li><span><a href="#Merge-&quot;duplicates&quot;-after-mapping" data-toc-modified-id="Merge-&quot;duplicates&quot;-after-mapping-3.1.9">Merge "duplicates" after mapping</a></span></li><li><span><a href="#EDA-generating-TTD-links" data-toc-modified-id="EDA-generating-TTD-links-3.1.10">EDA generating TTD links</a></span></li></ul></li><li><span><a href="#P1-07:-&quot;drug-affects-protein&quot;" data-toc-modified-id="P1-07:-&quot;drug-affects-protein&quot;-3.2">P1-07: "drug affects protein"</a></span><ul class="toc-item"><li><span><a href="#Load-file" data-toc-modified-id="Load-file-3.2.1">Load file</a></span></li><li><span><a href="#EDA" data-toc-modified-id="EDA-3.2.2">EDA</a></span></li><li><span><a href="#MOA-parsing" data-toc-modified-id="MOA-parsing-3.2.3">MOA parsing</a></span></li><li><span><a href="#Map-TTD-drug-IDs" data-toc-modified-id="Map-TTD-drug-IDs-3.2.4">Map TTD drug IDs</a></span></li><li><span><a href="#Map-TTD-target-IDs" data-toc-modified-id="Map-TTD-target-IDs-3.2.5">Map TTD target IDs</a></span></li><li><span><a href="#Merge-&quot;duplicates&quot;-after-mapping" data-toc-modified-id="Merge-&quot;duplicates&quot;-after-mapping-3.2.6">Merge "duplicates" after mapping</a></span></li><li><span><a href="#EDA-generating-TTD-links" data-toc-modified-id="EDA-generating-TTD-links-3.2.7">EDA generating TTD links</a></span></li><li><span><a href="#For-comparing-to-pipeline-output" data-toc-modified-id="For-comparing-to-pipeline-output-3.2.8">For comparing to pipeline output</a></span></li></ul></li></ul></li></ul></div>

# TTD notebook: testing code, EDA

In [1]:
## for notebook only 

## allows multiple lines of code to print from one code block
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## for printing
from pprint import pprint

## for loading locally-stored files
import pathlib

In [2]:
## INCLUDE in parser

import re
from typing import Dict, Union, Iterable
import itertools
import pandas as pd
import requests
import openpyxl  ## to check if environ has, needed for pd.read_excel

## from BioThings annotator code: for interoperability between diff Python versions
try:
    from itertools import batched  # new in Python 3.12
except ImportError:
    from itertools import islice

    def batched(iterable, n):
        # batched('ABCDEFG', 3) → ABC DEF G
        if n < 1:
            raise ValueError("n must be at least one")
        iterator = iter(iterable)
        while batch := tuple(islice(iterator, n)):
            yield batch


## NameRes dev url
NAMERES_URL = "https://name-resolution-sri.renci.org/bulk-lookup"
            
    
## paths to raw resource files
base_file_path = pathlib.Path.home().joinpath("Desktop", "TTD_files")
            
## NOT for parser: for viewing df only
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)

<div class="alert alert-block alert-danger">

This notebook was originally written using resource files downloaded 2025-10-30 from https://ttd.idrblab.net/full-data-download. 

## Parse header - define function!

First step is parsing the header ONLY to get:
* its line count - then we know how many lines to skip to get to the actual data
* version/date of this specific file, included in the header

The header ends with a 2nd dash "divider" line: either "----" or "____". 

We want this to be a separate function so it's reusable for all "custom txt" files with this kind of header. 

In [3]:
## regex testing

ex_string = "Version 10.1.01 (2024.01.10)"

pattern = "^Version ([0-9\\.]+) \\(([0-9\\.]+)\\)"

a = re.search(pattern, ex_string)
a.group(1)
a.group(2)

'10.1.01'

'2024.01.10'

In [4]:
## INCLUDE in parser

def parse_header(file_path) -> Dict[str, Union[str, int]]:
    ## use \\ to escape special characters like ".", "()"
    version_pattern = "^Version ([0-9\\.]+) \\(([0-9\\.]+)\\)"
    
    line_counter = 0  ## count lines read so far
    dash_counter = 0  ## count dash "divider" lines - header ends after 2nd one

    with open(file_path, 'r') as file:
        for line in file:
            if dash_counter == 2:  ## already read the 2nd "divider" line
                break
            else:
                line_counter += 1
                ## if line is a dash "divider" line
                if line.startswith("---") or line.startswith("___"):
                    dash_counter += 1
                ## assuming there's only 1 line in the header that matches this condition
                elif line.startswith("Version"):
                    capture = re.search(version_pattern, line)
                    version = capture.group(1)
                    date = capture.group(2)
                    date = date.replace(".", "-")
    
    return {
        "len_header": line_counter,
        "version": version,
        "date": date
    }

# ID-mapping 

## P1-03: TTD Drug IDs -> PUBCHEM.COMPOUND, CAS, CHEBI

### Parse header

In [87]:
## paths to raw resource files

p1_03_path = base_file_path.joinpath("P1-03-TTD_crossmatching.txt")

In [88]:
## INCLUDE in parser

p1_03_header_info = parse_header(p1_03_path)
p1_03_header_info

{'len_header': 27, 'version': '10.1.01', 'date': '2024-01-10'}

### EDA parse mappings

* Uses [itertools.islice](https://stackoverflow.com/a/53942825) ([Python](https://docs.python.org/3/library/itertools.html#itertools.islice) [docs](https://coderivers.org/blog/python-islice/)) and string method removeprefix (from [python 3.9+](https://www.geeksforgeeks.org/python/python-string-removeprefix-function/))
* the order of namespace lines should be PUBCHCID, CASNUMBE, CHEBI_ID (aka PUBCHCID is always first and could create the TTD ID entry). But I'm not assuming this, so for PUBCHCID, I'm always checking if the TTD ID entry already exists or not. 
* assuming that there is 1 of each namespace line in 1 TTD ID entry. If this assumption is false, this code will only keep the latest encounter's data. 
* the "adding to mapping dict" logic is the same for each namespace. BUT not making into a function because it would involve passing the dict in and out/rewriting it which would cost more

In [7]:
## format {TTD: {namespace: [list of IDs with correct prefixes]}}
temp_drug_mappings = dict()

## dev: stopping loop
# line_counter = 0

with open(p1_03_path, 'r') as file:
    ## iterate from beginning of data (after 2nd dash divider line) to end of file
    for line in itertools.islice(file, p1_03_header_info["len_header"], None):
        
        ## skip "blank" lines that only contain whitespace (seem to be "\n")
        if line.isspace():
            continue
        else:
            ## tab-delimited, line ends in "\n" so remove whitespace
            ## [0] == TTD ID, [1] == column name, [2] == value
            data = [i for i in line.strip().split("\t")]
            
            ## grab namespace lines
            if data[1] == "PUBCHCID":
                ## "value" can be "; "-delimited
                ## there can be empty values after spliting (ex: splitting "91865905; ")
                ##   using if to remove these cases
                ## And its ids don't have prefixes, so add desired prefix
                pubchem_ids = ["PUBCHEM.COMPOUND:" + j.strip() for j in data[2].split(";") if j.strip()]
                ## add to temp_drug_mappings
                ## if TTD ID isn't in mapping dict yet
                if data[0] not in temp_drug_mappings.keys():
                    temp_drug_mappings[data[0]] = {"pubchem_compound": pubchem_ids}
                ## already in mapping dict
                else:
                    temp_drug_mappings[data[0]]["pubchem_compound"] = pubchem_ids
            elif data[1] == "CASNUMBE":
                ## for EDA only
                if not data[2].startswith("CAS "):   
                    print(data[0], data[2])
                    
                ## based on grep EDA, this value is always single (no delimiter)
                ## remove prefix if it exists, then add desired prefix
                ## not doing a replace because some lines don't have "CAS "
                cas_id = "CAS:" + data[2].removeprefix("CAS ")
                ## add to temp_drug_mappings
                ## if TTD ID isn't in mapping dict yet
                if data[0] not in temp_drug_mappings.keys():
                    temp_drug_mappings[data[0]] = {"cas": [cas_id]}
                ## already in mapping dict
                else:
                    temp_drug_mappings[data[0]]["cas"] = [cas_id]
            elif data[1] == "CHEBI_ID":
                ## based on grep EDA, this value is always single (no delimiter)
                ## and the prefix is already desired format
                ## add to temp_drug_mappings
                ## if TTD ID isn't in mapping dict yet
                if data[0] not in temp_drug_mappings.keys():
                    temp_drug_mappings[data[0]] = {"chebi": [data[2]]}
                ## already in mapping dict
                else:
                    temp_drug_mappings[data[0]]["chebi"] = [data[2]]
                    
#         line_counter += 1
#         if line_counter > 20:
#             pprint(temp_drug_mappings)
#             break

D0A9ZX 6834-98-6
D0LR4B 68767-14-6
D0QO7H 1143-38-0
D0S0ES 89371-37-9
D0W9GA 78994-23-7
D02HUB 7759-35-5
D0W2FZ 760981-83-7
D00SBD 203191-10-0
D0E9BF 67-47-0
D0K4MU 221019-25-6
D0G6HI 239802-15-4
D0PK3W 169148-84-9
D0Q3QS 133-10-8
D0S1BN 642407-44-1
D0V9LJ 156953-42-3
D0E4FG 174391-92-5
D0LK8O 119514-66-8
D05MEY 1203902-67-3
D08RUN 18263-25-7
D09MXM 88939-40-6
D0NZ3Q 339615-76-8
D0Y0ID 3131-60-0


In [8]:
## check mappings

## not expected CAS format - handled correctly
# temp_drug_mappings["D0K4MU"]

## multiple pubchem ids
# multi_counter = 0
# for k,v in temp_drug_mappings.items():
#     if v.get("pubchem_compound"):
#         if len(v["pubchem_compound"]) > 1:
#             multi_counter += 1
# #             print(k,v)
# #             break
# multi_counter

# ## has chebi id
# for k,v in temp_drug_mappings.items():
#     if v.get("chebi"):
#         print(k,v)
#         break

## check that this doesn't have empty string ID - "PUBCHEM.COMPOUND:"
# temp_drug_mappings["D0B0HK"]
    
    
## look for entries without namespace: 
## actually, all have pubchem_compound IDs o_0 - nothing prints
for k,v in temp_drug_mappings.items():
    if not v.get("pubchem_compound"):
        print(k,v)

In [9]:
## how many mappings total / for each namespace

print(f"Total TTD drug IDs mapped: {len(temp_drug_mappings)}\n")

pubchem_counter = 0
cas_counter = 0
chebi_counter = 0

for k,v in temp_drug_mappings.items():
    if v.get("pubchem_compound"):
        pubchem_counter += 1
    if v.get("cas"):
        cas_counter += 1
    if v.get("chebi"):
        chebi_counter += 1
        
print(f"pubchem mappings: {pubchem_counter}")
print(f"cas mappings: {cas_counter}")
print(f"chebi mappings: {chebi_counter}")

Total TTD drug IDs mapped: 25423

pubchem mappings: 25423
cas mappings: 12163
chebi mappings: 5231


**So...we could use only the pubchem mappings!**

Now double-checking if there are cases of multiple PUBCHCID lines for 1 TTD ID (there shouldn't be). If we don't have to worry about this, then this makes final parsing simpler

In [10]:
## diff format than EDA! {TTD: [list of pubchem compound IDs with correct prefixes]}
temp_drug_mappings = dict()

## dev: stopping loop
# line_counter = 0

with open(p1_03_path, 'r') as file:
    ## iterate from beginning of data (after 2nd dash divider line) to end of file
    for line in itertools.islice(file, p1_03_header_info["len_header"], None):
        
        ## skip "blank" lines that only contain whitespace (seem to be "\n")
        if line.isspace():
            continue
        else:
            ## tab-delimited, line ends in "\n" so remove whitespace
            ## [0] == TTD ID, [1] == column name, [2] == value
            data = [i for i in line.strip().split("\t")]
            
            ## grab pubchem-compound lines
            if data[1] == "PUBCHCID":
                ## "value" can be "; "-delimited.
                ## there can be empty values after spliting (ex: splitting "91865905; ")
                ##   using if to remove these cases
                ## And its ids don't have prefixes, so add desired prefix
                pubchem_ids = ["PUBCHEM.COMPOUND:" + j.strip() for j in data[2].split(";") if j.strip()]
                
                ## add to mappings
                ## if TTD ID isn't in mapping dict yet
                if data[0] not in temp_drug_mappings.keys():
                    temp_drug_mappings[data[0]] = pubchem_ids
                ## already in mapping dict - catch this!!
                else:
                    ## debug
                    print(data[0])
#                     ## logic to add info/get unique values only
#                     temp_drug_mappings[data[0]].append(pubchem_ids) 
#                     temp_drug_mappings[data[0]] = list(set(temp_drug_mappings[data[0]]))
                    
#         line_counter += 1
#         if line_counter > 20:
#             pprint(temp_drug_mappings)
#             break

Nothing prints, so **there's no cases of multiple PUBCHCID lines for 1 TTD ID**!

<div class="alert alert-block alert-success">

**FOR PIPELINE CODE**: 
* only grab **pubchem_compound** mapping (Other namespaces don't increase the number of mapped TTD IDs)
* assume there's 1 PUBCHCID line per TTD ID

### Parser code

In [89]:
## INCLUDE in parser

## format {TTD: [list of pubchem compound IDs with correct prefixes]}
ttd_drug_mappings = dict()

## dev: stopping loop
# line_counter = 0

with open(p1_03_path, 'r') as file:
    ## iterate from beginning of data (after 2nd dash divider line) to end of file
    for line in itertools.islice(file, p1_03_header_info["len_header"], None):
        
        ## skip "blank" lines that only contain whitespace (seem to be "\n")
        if line.isspace():
            continue
        else:
            ## tab-delimited, line ends in "\n" so remove whitespace
            ## [0] == TTD ID, [1] == column name, [2] == value
            data = [i for i in line.strip().split("\t")]
            
            ## grab pubchem-compound lines
            if data[1] == "PUBCHCID":
                ## "value" can be "; "-delimited.
                ## there can be empty values after splitting (ex: splitting "91865905; ")
                ##   using if to remove these cases
                ## And its ids don't have prefixes, so add desired prefix
                pubchem_ids = ["PUBCHEM.COMPOUND:" + j.strip() for j in data[2].split(";") if j.strip()]
                
                ## add to mappings
                ttd_drug_mappings[data[0]] = pubchem_ids
                
#         line_counter += 1
#         if line_counter > 20:
#             pprint(ttd_drug_mappings)
#             break

In [90]:
len(ttd_drug_mappings)

ttd_drug_mappings["D0B0HK"]

25423

['PUBCHEM.COMPOUND:91865905']

## P2-01: TTD Target IDs -> UniProtKB names -> IDs

### Parse header

In [5]:
## paths to raw resource files

p2_01_path = base_file_path.joinpath("P2-01-TTD_uniprot_all.txt")

In [6]:
## INCLUDE in parser

p2_01_header_info = parse_header(p2_01_path)
p2_01_header_info

{'len_header': 22, 'version': '10.1.01', 'date': '2024-01-10'}

### EDA grab content

**P2-01 provides mappings to UniProt NAMES, not IDs.**

* inspired by Lucy's parser logic in [mapping_drug_id](https://github.com/lucyzhang95/BioThings_TTD_Dataplugin/blob/3b72b79b09a2f24fec6c29273df66c20c89590e3/TTD_parser.py#L259) and my previous work with P1-03 and P1-05
* Uses [itertools.islice](https://stackoverflow.com/a/53942825) ([Python](https://docs.python.org/3/library/itertools.html#itertools.islice) [docs](https://coderivers.org/blog/python-islice/)) and string method removeprefix (from [python 3.9+](https://www.geeksforgeeks.org/python/python-string-removeprefix-function/))
* assuming each TARGETID line has 1 corresponding UNIPROID line (their grep counts are the same). If this assumption is false, this code will only keep the latest encounter's data.
* Not using the TTD target name (TARGNAME) because after trying some out manually, NameRes doesn't reliably find the correct entity

In [7]:
## format {TTD: {"uniprot_names": [list]}}
temp_target_mappings = dict()
## get set of all unique names for mapping step
temp_names = set()
## get set of all odd-character sets to review, may be delimiters or invalid uniprot names
## "\W" catch any character that ISN'T alpha-numeric or underscore
odd_char_sets = set()
odd_pat = r"[\W]+"


## dev: stopping loop
line_counter = 0

with open(p2_01_path, 'r') as file:
    ## iterate from beginning of data (after 2nd dash divider line) to end of file
    for line in itertools.islice(file, p2_01_header_info["len_header"], None):
        ## skip "blank" lines that only contain whitespace (seem to be "\n")
        ## these tend to be between TTD ID entries, so reset temp variable
        if line.isspace():
            line_counter += 1
            ttd_target = ""
            continue
        else:
            ## tab-delimited, has extra whitespace at end of line
            data = [i for i in line.strip().split("\t")]
            
            ## TTD target ID line
            if data[0] == "TARGETID":
                ## save in temp variable, always seems to be single value
                ttd_target = data[1]
            elif data[0] == "UNIPROID":
                ## EDA: if ttd_target is empty, "TARGETID" didn't come first...needs review
                ## if this never triggers, we don't need this layer of if-else in parser
                if not ttd_target:
                    print(line_counter)
                    print(data)
                    continue
                else:
                    ## don't save name if it seems fake
                    ## NOUNIPROTAC seems to be "NO UNIPROT AC"
                    if data[1] != "NOUNIPROTAC":
                        ## check for odd strings, unexpected in uniprot name
                        odd_finds = re.findall(odd_pat, data[1])
                        odd_char_sets.update(odd_finds)
                        
                        ## split on odd characters (>=1 together) (same as findall)
                        ## only save element if it isn't an empty string (if statement)
                        uniprot_names = [j for j in re.split(odd_pat, data[1]) if j.strip()]
                        ## add to unique set
                        temp_names.update(uniprot_names)

                        ## add to temp_target_mappings
                        ## if this never triggers, we don't need this layer of if-else in parser
                        ## if TTD ID isn't in mapping dict yet
                        if ttd_target not in temp_target_mappings.keys():
                            temp_target_mappings[ttd_target] = {"uniprot_names": uniprot_names}
                        ## already in mapping dict
                        else:
                            print(f"TTD ID already in mapping dict: {ttd_target}")
                            temp_drug_mappings[ttd_target]["uniprot_names"].append(uniprot_names)
                            temp_drug_mappings[ttd_target]["uniprot_names"] = list(set(temp_drug_mappings[ttd_target]["uniprot_names"]))
                ## reset temp TTD ID to keep track
                ttd_target = ""
                        
                    
        line_counter += 1
#         if line_counter > 20:
#             pprint(temp_target_mappings)
#             break

Nothing prints, so there's no cases where:
* TARGETID didn't come first/wasn't present
* there were multiple UNIPROID entries for the same TARGETID

**So we can remove those if-else layers from the parser code and resetting the TTD ID** (didn't encounter any cases where the reset caught a problem)

In [8]:
## expected number based on grep counts and calculations!
len(temp_target_mappings)

len(temp_names)

3685

3513

This number is less than the number of mappings. And we know some mappings have multiple names. 

**This means there's a good number of uniprot names that show up in multiple TTD ID entries.** 

In [9]:
## expected number: count lines - header
line_counter

## finding examples of multiple values using grep
temp_target_mappings["T02258"]

21490

{'uniprot_names': ['DPOLQ_HUMAN',
  'DPOLN_HUMAN',
  'DPOLB_HUMAN',
  'DPOG2_HUMAN',
  'DPOG1_HUMAN',
  'DPOE2_HUMAN',
  'DPOD4_HUMAN',
  'DPOD1_HUMAN']}

In [10]:
## check for duplicates within an entry
for k,v in temp_target_mappings.items():
    n_before = len(v["uniprot_names"])
    if n_before > 1:
        n_after = len(set(v["uniprot_names"]))
        if n_before != n_after:
            print(k,v)

Nothing prints for second block, so **there are no duplicates in an entry and parser code doesn't need to do `list(set(x))` to deduplicate**. 

In [11]:
odd_char_sets

{' ', ' (', ')', '-', '/', '; '}

**Investigation using grep**

`' '` and `' ('` and `')'` current line matches: `TAT_HV1H2-TAR RNA` and `POLG_DEN1W (281-775)`
* **DECISION**: only save j (element from delimiter split) if these strings not in it (pandas is where "contains" method is) 
* this means these aren't valid uniprot names
* grep to find word+space: `grep UNIPROID P2-01-TTD_uniprot_all.txt | grep -E "\w+ "`)


**DELIMITERS**:
* `'-'`: practically always (see 1 exception above)
* `'/'`: 3 current line matches
* `'; '`: 118 current line matches

### Parser - grab content

Now remove unneeded if/else logic and add special char logic

In [17]:
## INCLUDE in parser

## format {TTD: {"uniprot_names": [list]}}
ttd_target_mappings = dict()
## get set of all unique names for mapping step
all_uniprot_names = set()
## saved for this function
DELIMITER_PAT = "-|/|;"
INVALID_CHAR = [" ", "(", ")"]


# ## dev: stopping loop
# line_counter = 0

with open(p2_01_path, 'r') as file:
    ## iterate from beginning of data (after 2nd dash divider line) to end of file
    for line in itertools.islice(file, p2_01_header_info["len_header"], None):
        ## skip "blank" lines that only contain whitespace (seem to be "\n")
        if line.isspace():
#             line_counter += 1
            continue
        else:
            ## tab-delimited, may have extra whitespace at end of line
            data = [i for i in line.strip().split("\t")]
            
            ## TTD target ID line
            if data[0] == "TARGETID":
                ## save in temp variable, always seems to be single value
                ttd_target = data[1]
            elif data[0] == "UNIPROID":
                ## don't save name if it seems fake
                ## NOUNIPROTAC seems to be "NO UNIPROT AC"
                if data[1] != "NOUNIPROTAC":
                    ## split on delimiter chars
                    ## only save element j if it is non-empty string and doesn't have any invalid char in it
                    ## strip surrounding whitespace off j and save
                    temp_name = [j.strip() \
                                     for j in re.split(DELIMITER_PAT, data[1]) \
                                     if j.strip() and not any(letter in INVALID_CHAR for letter in j.strip())]
                    
                    ## only add if has values
                    if temp_name:
                        ## add to unique set
                        all_uniprot_names.update(temp_name)
                        ## add to mappings
                        ttd_target_mappings[ttd_target] = {"uniprot_names": temp_name}
                        
                    
#         line_counter += 1
#         if line_counter > 20:
#             pprint(temp_target_mappings)
#             break

In [18]:
## EDA

len(ttd_target_mappings)
len(all_uniprot_names)

3684

3508

In [20]:
## EDA: looking at diff from previous parsing
set(temp_target_mappings.keys()) - set(ttd_target_mappings.keys())

temp_names - all_uniprot_names

all_uniprot_names - temp_names

{'T79401'}

{'281', '775', 'POLG_DEN1W', 'RNA', 'TAR'}

set()

In [25]:
## EDA - problematic lines

ttd_target_mappings["T56636"]

## have "-" and "/" delimiters
ttd_target_mappings["T15894"]
ttd_target_mappings["T30834"]

{'uniprot_names': ['TAT_HV1H2']}

{'uniprot_names': ['CUL4A_HUMAN', 'CUL4B_HUMAN', 'DDB1_HUMAN', 'CRBN_HUMAN']}

{'uniprot_names': ['CALCR_HUMAN', 'RAMP1_HUMAN', 'RAMP2_HUMAN', 'RAMP3_HUMAN']}

In [26]:
## EDA: check mappings

## number of TTD IDs with > 1 name mapping
## expected number based on grep counts of delimiters
multi_counter = 0
largest_n = 1
for k,v in temp_target_mappings.items():
    n_names = len(v["uniprot_names"])
    if n_names > 1:
        multi_counter += 1
    if n_names > largest_n:
        largest_n = n_names
#         print(k,v)
    if n_names > 7:
#         print(n_before)
        print(k,v)
#         break
multi_counter
## longest is 8 names 
largest_n

## any empty strings: if the trailing "if j.strip()" is removed
# for k,v in temp_target_mappings.items():
#     for i in v["uniprot_names"]:
#         ## if anything is an empty string
#         if not i:
#             print(k,v)
#             break
# #     ## need to break two layers empty string encountered to stop
# #     break

T02258 {'uniprot_names': ['DPOLQ_HUMAN', 'DPOLN_HUMAN', 'DPOLB_HUMAN', 'DPOG2_HUMAN', 'DPOG1_HUMAN', 'DPOE2_HUMAN', 'DPOD4_HUMAN', 'DPOD1_HUMAN']}


164

8

In [42]:
## EDA: to manually review some uniprot names

# sorted(all_uniprot_names)[3200:3300]

### Map Uniprot names -> IDs

Currently only using NameRes. 

Using UniProt is also an option. 

In [43]:
## function from parser

def run_nameres(names: Iterable[str], 
                url: str, 
                batch_size: int = 500, 
                types: list | None = None, 
                exclude_namespaces: str | None = None,
                score_threshold: int = 0,):
    """
    Parameters:
    - names: string names to NameRes (iterable - can be a set)
    - url: NameRes url/endpoint to use
    - batch_size: number of strings to include in 1 query to NameRes
    - types (default None): list of biolink categories that NameRes hits should have (hierarchy expansion is supported)
    - exclude_namespaces (default None): |-delimited string of ID namespaces to exclude, for quality
    - score_threshold (default 0): only accept hit if its score is greater than this, for quality

    Returns: tuple of mapping dict and failure stats dict
    """
    ## set up variables to collect output
    mapping = {}
    stats_failures = {
        "unexpected_error": {},
        "returned_empty": [],
        "score_under_threshold": [],
    }
#     ## for debug: stopping early
#     counter = 0

    for batch in batched(names, batch_size):
        req_body = {
            "strings": list(batch),  ## returns tuples -> cast to list
            "autocomplete": False,   ## names are complete search term
            "limit": 1,              ## only want to review top hit
            "biolink_types": types,
            "exclude_prefixes": exclude_namespaces,    ## try to increase quality of hits
        }    
        r = requests.post(url, json=req_body)
        response = r.json()

        ## not doing dict comprehension. allows easier review, logic writing
        for k,v in response.items():
            ## catch unexpected errors
            try:
                ## will catch if v is an empty list (aka NameRes didn't have info)
                if v:
                    ## v is a 1-element list, work with it directly
                    temp = v[0]
                    ## also throw out mapping if score < score_threshold: want better-matching hits
                    if temp["score"] > score_threshold:
                        mapping.update({
                            k: temp["curie"]
                        })
                    else:
                        stats_failures["score_under_threshold"].append(k)
                else:
                    stats_failures["returned_empty"].append(k)
            except Exception as e:
                stats_failures["unexpected_error"].update({k: e})

#         counter += batch_size
#         if counter >= 500:
#             break

    return mapping, stats_failures

In [57]:
# target_batch_size = 500
target_types = ["GeneOrGeneProduct"]
target_exclude_prefixes = "UMLS"

## to try with only_prefixes parameter in queries
# target_desired_namespaces = "UniProtKB|NCBIGene"
# target_score_threshold = 300


## use NAMERES_URL initialized earlier
uniprot_name_to_id, stats_target_mapping_failures = \
    run_nameres(
    names=all_uniprot_names,
    url=NAMERES_URL, 
#     batch_size=target_batch_size,
    types=target_types,
    exclude_namespaces=target_exclude_prefixes,
#     score_threshold=target_score_threshold,
    )

Testing diff batch sizes on first >=500:
* 100: 14s
* 200: 15s
* 300: 13s 
* **500: 11.5s**

Did try to use only_prefixes (since only want entities with UniProtKB, NCBIGene as prefix). But it would take much longer to run (ex: 3X longer)

In [62]:
## EDA only

len(uniprot_name_to_id)
len(uniprot_name_to_id) / len(all_uniprot_names)

stats_target_mapping_failures.keys()
stats_target_mapping_failures["unexpected_error"]

stats_target_mapping_failures["score_under_threshold"]

3452

0.984036488027366

dict_keys(['unexpected_error', 'returned_empty', 'score_under_threshold'])

{}

[]

In [59]:
## EDA only

len(stats_target_mapping_failures["returned_empty"])

## for review these manually
stats_target_mapping_failures["returned_empty"][0:10]

56

['G0TQ18_MYCCP',
 'O33336_MYCTU',
 '2B1A_HUMAN',
 'FAB1_MYCTU',
 'AMP1_PLAFQ',
 'GLRA4_HUMAN',
 'Q8I6Z5_PLAF7',
 'GLCM_HUMAN',
 'PORED_HUMAN',
 'LIPE_HUMAN']

In [79]:
## FUTURE

## save names that failed nameres
## in future, would run through uniprot website

target_name_failures = set(stats_target_mapping_failures["returned_empty"])
## just in case
target_name_failures.update(stats_target_mapping_failures["unexpected_error"].keys())
target_name_failures.update(stats_target_mapping_failures["score_under_threshold"])

len(target_name_failures)

56

Looked up some of these in UniProtKB. 

Situations: 
* UniProtKB finds current entry, this was an old name/old merged entry: 2B1A_HUMAN, AMP1_PLAFQ, CTGF_HUMAN
* moved to UniParc: G0TQ18_MYCCP, GLRA4_HUMAN
* no matches: O33336_MYCTU, FAB1_MYCTU

In [60]:
## EDA: manually reviewing some mappings

for i in itertools.islice(uniprot_name_to_id.items(), 0, 10):
    print(i)

('RPOB_STAA8', 'NCBIGene:3920377')
('SF3B5_HUMAN', 'NCBIGene:83443')
('SENX3_MYCTU', 'UniProtKB:P9WGK5')
('TRPV6_HUMAN', 'NCBIGene:55503')
('LPAR2_HUMAN', 'NCBIGene:9170')
('CTRO_HUMAN', 'NCBIGene:11113')
('ACKR1_HUMAN', 'NCBIGene:2532')
('MSH2_HUMAN', 'NCBIGene:4436')
('CD20_HUMAN', 'NCBIGene:931')
('SDC1_HUMAN', 'NCBIGene:6382')


In [61]:
## EDA: check - do all IDs start with UniProtKB or NCBIGene?
## want them to
for k,v in uniprot_name_to_id.items():
    if not (v.startswith("NCBIGene") or v.startswith("UniProtKB")):
        print(k,v)

## previous key with problematic higher-scoring UMLS match
uniprot_name_to_id["ERBB2_HUMAN"]

'NCBIGene:2064'

### Finish constructing mappings

In [73]:
## format {TTD: {"uniprot_names": [list], "mapped_ids": [list]}}

failed_TTD_targets = set()

for k,v in ttd_target_mappings.items():
    ## use set so no duplicate IDs
    temp_ids = set()
    for i in v["uniprot_names"]:
        temp = uniprot_name_to_id.get(i)
        ## found mapping
        if temp:
            temp_ids.add(temp)
    ## temp_ids isn't empty
    if temp_ids:
        ttd_target_mappings[k]["mapped_ids"] = list(temp_ids)
    else:
        failed_TTD_targets.add(k)

In [75]:
## number of TTD IDs with names but without ID mappings 

len(failed_TTD_targets)

55

In [80]:
len(ttd_target_mappings)

3684

In [81]:
## INCLUDE IN PARSER: delete TTD target mappings that don't have any IDs mapped

for i in failed_TTD_targets:
    del ttd_target_mappings[i]

In [82]:
len(ttd_target_mappings)

3629

In [86]:
## EDA    

# for i in itertools.islice(ttd_target_mappings.items(), 0, 10):
#     print(i)

## problematic ones
## "-", "/" delimited
ttd_target_mappings["T15894"]
ttd_target_mappings["T30834"]
## ";" delimited, longest
ttd_target_mappings["T02258"]

{'uniprot_names': ['CUL4A_HUMAN', 'CUL4B_HUMAN', 'DDB1_HUMAN', 'CRBN_HUMAN'],
 'mapped_ids': ['NCBIGene:51185',
  'NCBIGene:8450',
  'NCBIGene:8451',
  'NCBIGene:1642']}

{'uniprot_names': ['CALCR_HUMAN', 'RAMP1_HUMAN', 'RAMP2_HUMAN', 'RAMP3_HUMAN'],
 'mapped_ids': ['NCBIGene:10268',
  'NCBIGene:10266',
  'NCBIGene:799',
  'NCBIGene:10267']}

{'uniprot_names': ['DPOLQ_HUMAN',
  'DPOLN_HUMAN',
  'DPOLB_HUMAN',
  'DPOG2_HUMAN',
  'DPOG1_HUMAN',
  'DPOE2_HUMAN',
  'DPOD4_HUMAN',
  'DPOD1_HUMAN'],
 'mapped_ids': ['NCBIGene:5427',
  'NCBIGene:57804',
  'NCBIGene:5423',
  'NCBIGene:5424',
  'NCBIGene:10721',
  'NCBIGene:11232',
  'NCBIGene:5428',
  'NCBIGene:353497']}

# Edges

## P1-05: "drug treats indication"

### Parse header

In [291]:
p1_05_path = base_file_path.joinpath("P1-05-Drug_disease.txt")

In [292]:
## INCLUDE in parser

p1_05_header_info = parse_header(p1_05_path)
p1_05_header_info

{'len_header': 21, 'version': '10.1.01', 'date': '2024-03-30'}

### EDA grab content

* inspired by Lucy's parser logic in [load_drug_dis_data](https://github.com/lucyzhang95/BioThings_TTD_Dataplugin/blob/3b72b79b09a2f24fec6c29273df66c20c89590e3/TTD_parser.py#L350)
* putting rough edge objects into list, then load into pandas once - which is [recommended](https://sentry.io/answers/create-an-empty-python-pandas-dataframe-and-add-rows/)

**Using indication name "column" due to issues with ICD 11 "column" values:**
* is a code, not the "actual" ICD-11 ID aka "foundation" URI. Ex: TTD uses `5C58.03` for "Progressive familial intrahepatic cholestasis", but the [ICD-11 foundation URI is 1457142642](https://icd.who.int/browse/2025-01/mms/en#1457142642)
  * [MyDisease's mondo.xrefs.icd11](https://mydisease.info/v1/query?q=_exists_:mondo.xrefs.icd11&fields=mondo) use the "foundation" IDs, based on the ID format (numeric)
  * NodeNorm currently has very little support for ICD11 IDs in DiseaseOrPheno (5 total in database?), and they are the "foundation" URIs
* are sometimes a "range" of codes, not listed individually. Ex: for "solid tumour/cancer", TTD uses "2A00-2F9Z" aka code "2A00" to "2F9Z". 

In [293]:
## format [{edge objects}]
eda_edges = list()


## dev: stopping loop
line_counter = 0

with open(p1_05_path, 'r') as file:
    ## iterate from beginning of data (after 2nd dash divider line) to end of file
    for line in itertools.islice(file, p1_05_header_info["len_header"], None):
        ## skip "blank" lines that only contain whitespace (seem to be "\n")
        if line.isspace():
            ttd_drug = ""  ## reset temp variable
            continue
        else:
            ## tab-delimited, has extra whitespace at end of line
            data = [i for i in line.strip().split("\t")]
            
            ## TTD drug ID line
            if data[0] == "TTDDRUID":
                ## save in temp variable, always seems to be single value
                ttd_drug = data[1]
            elif data[0] == "INDICATI":
                ## EDA: if ttd_drug is empty, "TTDDRUID" didn't come first...needs review
                if not ttd_drug:
                    print(line_counter)
                    print(data)
                    continue
                else:
                    ## don't make edge if indication name seems fake
                    if data[1] == "#N/A":
                        continue
                    else:
                        eda_edges.append({
                            "subject_ttd_drug": ttd_drug,
                            "object_indication_name": data[1],
                            "clinical_status": data[3],
                        })
                    
        line_counter += 1
#         if line_counter > 20:
#             pprint(eda_edges)
#             break

Nothing printed - meaning we never had an empty ttd_drug variable when encountering an "indication" edge. 
This means we can simplify the code and assume that ttd_drug will always be filled properly before the "indication" rows. 

In [294]:
## basic EDA

## expected based on length of file, number of fake names
len(eda_edges)

eda_edges[100]

30312

{'subject_ttd_drug': 'D8LAE2',
 'object_indication_name': 'Non-small-cell lung cancer',
 'clinical_status': 'Phase 3'}

In [295]:
## EDA using dataframe

temp_df = pd.DataFrame(eda_edges)

## expected based on grep EDA, then subtract based on fake names
temp_df["subject_ttd_drug"].nunique()

temp_df["object_indication_name"].nunique()

temp_df["clinical_status"].nunique()

23717

1944

40

### Parser - grab content

In [296]:
## INCLUDE in parser

## format [{edge objects}]
edges = list()

with open(p1_05_path, 'r') as file:
    ## iterate from beginning of data (after 2nd dash divider line) to end of file
    for line in itertools.islice(file, p1_05_header_info["len_header"], None):
        ## skip "blank" lines that only contain whitespace (seem to be "\n")
        if line.isspace():
            continue
        else:
            ## tab-delimited, has extra whitespace at end of line
            data = [i for i in line.strip().split("\t")]
            
            ## TTD drug ID line
            if data[0] == "TTDDRUID":
                ## save in temp variable, always seems to be single value
                ttd_drug = data[1]
            elif data[0] == "INDICATI":
                ## don't make edge if indication name seems fake
                if data[1] == "#N/A":
                    continue
                else:
                    edges.append({
                        "subject_ttd_drug": ttd_drug,
                        "object_indication_name": data[1],
                        "clinical_status": data[3],
                    })

df = pd.DataFrame(edges)

In [297]:
## INCLUDE in parser -> log

print(f"{df.shape[0]} rows after parsing file\n")

print(f"{df["subject_ttd_drug"].nunique()} unique TTD drug IDs")

print(f"{df["object_indication_name"].nunique()} unique indication names")

print(f"{df["clinical_status"].nunique()} unique clinical status values")

30312 rows after parsing file

23717 unique TTD drug IDs
1944 unique indication names
40 unique clinical status values


In [298]:
df

,subject_ttd_drug,object_indication_name,clinical_status
0,DZB84T,Pruritus,Approved
1,DZB84T,Progressive familial intrahepatic cholestasis,Phase 3
2,DZB84T,Alagille syndrome,Phase 2
3,DZA90G,Coronavirus Disease 2019 (COVID-19),Approved
4,DZ8DF0,Primary hyperoxaluria type 1,Approved
...,...,...,...
30307,D00AJS,Non-insulin dependent diabetes,Investigative
30308,D00AIZ,Mycobacterium infection,Investigative
30309,D00AIS,Solid tumour/cancer,Investigative
30310,D00AHV,Central nervous system disease,Investigative


### Map clinical_status

"clinical status" values -> biolink predicate ("treats" hierarchy)

Doing first so we can review all possible values.

<div class="alert alert-block alert-danger">

**Values not included in `CLINICAL_STATUS_MAP` will be filtered out. This means new values need to be manually reviewed, added to `CLINICAL_STATUS_MAP` if we want to keep their data.**

In [18]:
## EDA

df["clinical_status"].nunique()

df["clinical_status"].value_counts().sort_index()

## to get without counts
# sorted(df["clinical_status"].unique())

40

clinical_status
Application submitted                72
Approval submitted                    1
Approved                           3742
Approved (orphan drug)                5
Approved in China                     5
Approved in EU                        8
BLA submitted                         1
Clinical Trial                        7
Clinical trial                      108
Discontinued in Phase 1             745
Discontinued in Phase 1/2            57
Discontinued in Phase 2            1091
Discontinued in Phase 2/3             6
Discontinued in Phase 2a              1
Discontinued in Phase 2b              2
Discontinued in Phase 3             268
Discontinued in Phase 4               2
Discontinued in Preregistration      41
IND submitted                         7
Investigative                      4619
NDA filed                             6
Patented                           3155
Phase 0                              11
Phase 1                            4956
Phase 1/2               

In [299]:
## INCLUDE in parser

## biolink predicates
BIOLINK_TREATS = "biolink:treats"
BIOLINK_STUDIED_TREAT = "biolink:studied_to_treat"
BIOLINK_PRECLINICAL = "biolink:in_preclinical_trials_for"
BIOLINK_CLINICAL_TRIALS = "biolink:in_clinical_trials_for"

## hard-coded mapping of "clinical status" values to biolink "treats" predicates
## purposely doesn't include all values - rest are filtered out
CLINICAL_STATUS_MAP = {
    ## treats
    "Approved": BIOLINK_TREATS,
    "Approved (orphan drug)": BIOLINK_TREATS,
    "Approved in China": BIOLINK_TREATS,
    "Approved in EU": BIOLINK_TREATS,
    "Phase 4": BIOLINK_TREATS,
    ## studied to treat
    "Investigative": BIOLINK_STUDIED_TREAT,
    "Patented": BIOLINK_STUDIED_TREAT,
    ## in preclinical trials for
    "Preclinical": BIOLINK_PRECLINICAL,
    "IND submitted": BIOLINK_PRECLINICAL,
    ## in clinical trials for
    "Clinical Trial": BIOLINK_CLINICAL_TRIALS,
    "Clinical trial": BIOLINK_CLINICAL_TRIALS,
    "Preregistration": BIOLINK_CLINICAL_TRIALS,
    "Registered": BIOLINK_CLINICAL_TRIALS,
    "Phase 0": BIOLINK_CLINICAL_TRIALS,
    "Phase 1": BIOLINK_CLINICAL_TRIALS,
    "Phase 1b": BIOLINK_CLINICAL_TRIALS,
    "Phase 1/2": BIOLINK_CLINICAL_TRIALS,
    "Phase 1/2a": BIOLINK_CLINICAL_TRIALS,
    "Phase 1b/2a": BIOLINK_CLINICAL_TRIALS,
    "Phase 2": BIOLINK_CLINICAL_TRIALS,
    "Phase 2a": BIOLINK_CLINICAL_TRIALS,
    "Phase 2b": BIOLINK_CLINICAL_TRIALS,
    "Phase 2/3": BIOLINK_CLINICAL_TRIALS,
    "Phase 3": BIOLINK_CLINICAL_TRIALS,
    "phase 3": BIOLINK_CLINICAL_TRIALS,
    "NDA filed": BIOLINK_CLINICAL_TRIALS,
    "BLA submitted": BIOLINK_CLINICAL_TRIALS,
    "Approval submitted": BIOLINK_CLINICAL_TRIALS,
}

**[last edited 2025-11-03, based on v10.1.01 '2024-03-30' data]**

**NOTES ON MAPPING**
* treats:
  * `Phase 4` is after the drug is approved by FDA - [ref](https://www.allclinicaltrials.com/blog/phases-of-clinical-trials)
* studied_to_treat:
  * `Investigative`: not sure what this exactly means. VS [investigational](https://www.cancer.gov/publications/dictionaries/cancer-terms/def/investigational-drug) is a formal term meaning in clinical trials
  * `Patented`: according to [TTD 2020 paper](https://academic.oup.com/nar/article/48/D1/D1031/5613683#190994680), these are in "early drug discovery" stage and include "potential therapeutic indications"
* in_preclinical_trials_for:
  * `IND submitted`: application to start clinical trials - ref [gov](https://www.cc.nih.gov/orcs/ind), [fdareview](https://www.fdareview.org/issues/the-drug-development-and-approval-process/), [allucent](https://www.allucent.com/resources/blog/5-common-fda-applications-drugs-and-biologics)
* in_clinical_trials_for - [ref](https://www.allclinicaltrials.com/blog/phases-of-clinical-trials)
  * `Preregistration`: registering clinical trial before/at start of clinical trial - [ref](https://www.atsjournals.org/doi/10.1513/AnnalsATS.202508-835ED)
  * `Registered`: not sure what this exactly means. Assuming it's "clinical trial registered"
  * `NDA filed`: applied for FDA approval of drug after "finishing" clinical trials - ref [fdareview](https://www.fdareview.org/issues/the-drug-development-and-approval-process/), 
  * `BLA submitted`: applied for FDA approval of biologic after "finishing" clinical trials - ref [allucent](https://www.allucent.com/resources/blog/what-are-regulatory-differences-between-nda-and-bla)
  * `Approval submitted`: not sure what this exactly means. Assuming it's the "drug approval" application after clinical trials done and successful
  

**Filter out everything else**

I was unsure, considered its ingest "controversial" (people would have diff opinions):
* `Application submitted`: not sure what this exactly means - "application" is too vague
* All the "discontinued" values: this can happen for reasons not related to safety/efficacy - [ref](https://doi.org/10.1186/s13104-020-05391-w)
  * `Discontinued in Preregistration`
  * `Discontinued in Phase 1`
  * `Discontinued in Phase 1/2`
  * `Discontinued in Phase 2`
  * `Discontinued in Phase 2a`
  * `Discontinued in Phase 2b`
  * `Discontinued in Phase 2/3`
  * `Discontinued in Phase 3`
  * `Discontinued in Phase 4`
* `Terminated`: can happen for reasons not related to safety/efficacy - [ref](https://pmc.ncbi.nlm.nih.gov/articles/PMC4444136/)
* `Withdrawn from market`

**FUTURE:**
* review filtered-out terms, figure out how to ingest them (predicate to use)? 
* mappings to edge attribute(s) values that capture original "clinical status" meaning more precisely


**CONCERNS:**

**Some P1-05 data doesn't actually fit "treats".** Ex: indication is "contraception" or drug is for imaging (SomaKit-TOC; gallium (68Ga) edotreotide). 

**TTD actually uses "indication", which seems to be a wider term than "treats…".** <br>
Seems more like "reasonable to use for condition X" or "has therapeutic use for condition X"

* Includes "diagnosis" and "used to achieve" (ex: contraception)
* Includes things that aren't diseases - sign, symptom, condition
* [Wiki](https://en.wikipedia.org/wiki/Indication_(medicine)): "**a valid reason to use** a certain **test**, medication, procedure, or surgery." 
  * Opposite is contraindication ("a reason to withhold"). 
  * Drug can be **indicated for** "treatment, **prevention, mitigation, cure, relief, or diagnosis** of that disease or **condition**"
  * There's also the official "drug indications" approved by licensing body 
* [NCI](https://www.cancer.gov/publications/dictionaries/cancer-terms/def/indication): "a **sign, symptom, or medical condition** that leads to the recommendation of a treatment, **test, or procedure**."
* [Oprea/Dumontier/etc paper](https://pmc.ncbi.nlm.nih.gov/articles/PMC6259666/): "Indications" and actually describing the therapeutic intent are tricky! 


In [300]:
## INCLUDE in parser

## map "clinical status" to biolink predicate
## get method returns None if key (clinical status) not found in mapping 
df["biolink_predicate"] = [CLINICAL_STATUS_MAP.get(i) for i in df["clinical_status"]]

In [301]:
## INCLUDE in parser -> log

n_mapped = df["biolink_predicate"].notna().sum()
print(f"{n_mapped} rows with mapped clinical status: {n_mapped / df.shape[0]:.1%}")

26474 rows with mapped clinical status: 87.3%


**~87% of the data could be mapped.**

In [302]:
## INCLUDE in parser -> log/save

## double-checking what clinical status values aren't mapped
sorted(df[df["biolink_predicate"].isna()].clinical_status.unique())

['Application submitted',
 'Discontinued in Phase 1',
 'Discontinued in Phase 1/2',
 'Discontinued in Phase 2',
 'Discontinued in Phase 2/3',
 'Discontinued in Phase 2a',
 'Discontinued in Phase 2b',
 'Discontinued in Phase 3',
 'Discontinued in Phase 4',
 'Discontinued in Preregistration',
 'Terminated',
 'Withdrawn from market']

In [303]:
## INCLUDE in parser

## drop rows without predicate mapping
df.dropna(subset="biolink_predicate", inplace=True, ignore_index=True)

In [304]:
## EDA

df.shape

(26474, 4)

### Map TTD drug IDs

Uses `ttd_drug_mappings` generated earlier when parsing mapping files (P1-03). 

If mappings had more namespaces (logic needed to pick one), would write it out - not use list comprehension:
* If getting TTD ID successful...
  * If getting pubchem successful...use it
  * Else, use inchikey
 * Else: use None

In [305]:
## INCLUDE in parser

## get method returns None if key (TTD ID) not found in mapping 
df["subject_pubchem"] = [ttd_drug_mappings.get(i) for i in df["subject_ttd_drug"]]

In [306]:
## EDA - check that no value is "PUBCHEM.COMPOUND:"

df[df["subject_pubchem"] == "PUBCHEM.COMPOUND:"]

,subject_ttd_drug,object_indication_name,clinical_status,biolink_predicate,subject_pubchem


In [307]:
## INCLUDE in parser -> log

n_mapped = df["subject_pubchem"].notna().sum()
print(f"{n_mapped} rows with mapped TTD drug IDs: {n_mapped / df.shape[0]:.1%}")

10148 rows with mapped TTD drug IDs: 38.3%


**Only ~38% of the data could be mapped.**

In [308]:
## INCLUDE in parser

## remove rows without drug mapping
df.dropna(subset="subject_pubchem", inplace=True, ignore_index=True)

In [309]:
df.shape

(10148, 5)

In [310]:
## EDA: multiple pubchem values? 

df[df["subject_pubchem"].map(len) > 1].shape

df[df["subject_pubchem"].map(len) > 1]

(37, 5)

,subject_ttd_drug,object_indication_name,clinical_status,biolink_predicate,subject_pubchem
185,D0XW2H,Asthma,Approved,biolink:treats,"[PUBCHEM.COMPOUND:441336, PUBCHEM.COMPOUND:3083544]"
1690,D0C4DH,Hepatitis C virus infection,Approved,biolink:treats,"[PUBCHEM.COMPOUND:66828839, PUBCHEM.COMPOUND:58031952]"
2254,D07DPI,Cardiovascular disease,Approved,biolink:treats,"[PUBCHEM.COMPOUND:2244, PUBCHEM.COMPOUND:9568614]"
2429,D06AGH,Melanoma,Approved,biolink:treats,"[PUBCHEM.COMPOUND:11707110, PUBCHEM.COMPOUND:44462760]"
2508,D05JFY,Heart failure,Approved,biolink:treats,"[PUBCHEM.COMPOUND:9351, PUBCHEM.COMPOUND:6883]"
3424,D0Z1SL,Postmenopausal osteoporosis,Phase 3,biolink:in_clinical_trials_for,"[PUBCHEM.COMPOUND:154257, PUBCHEM.COMPOUND:45357473]"
3426,D0Z0HL,Type-2 diabetes,Phase 3,biolink:in_clinical_trials_for,"[PUBCHEM.COMPOUND:90472060, PUBCHEM.COMPOUND:44146714]"
3692,D0L4LX,Hypercholesterolaemia,Phase 3,biolink:in_clinical_trials_for,"[PUBCHEM.COMPOUND:60823, PUBCHEM.COMPOUND:150311]"
3794,D0H0UB,Chronic obstructive pulmonary disease,Phase 3,biolink:in_clinical_trials_for,"[PUBCHEM.COMPOUND:25195533, PUBCHEM.COMPOUND:642444]"
3949,D0C0MN,Hypertension,Phase 3,biolink:in_clinical_trials_for,"[PUBCHEM.COMPOUND:3749, PUBCHEM.COMPOUND:5560]"


In [311]:
## INCLUDE in parser

## expand to multiple rows when subject_pubchem list length > 1
## also pops every subject_pubchem value out into a string
df = df.explode("subject_pubchem", ignore_index=True)

## -> log
print(f"{df.shape[0]} rows after expanding mappings with multiple ID values")

## EDA only
df

10190 rows after expanding mappings with multiple ID values


,subject_ttd_drug,object_indication_name,clinical_status,biolink_predicate,subject_pubchem
0,DZB84T,Pruritus,Approved,biolink:treats,PUBCHEM.COMPOUND:9831643
1,DZB84T,Progressive familial intrahepatic cholestasis,Phase 3,biolink:in_clinical_trials_for,PUBCHEM.COMPOUND:9831643
2,DZB84T,Alagille syndrome,Phase 2,biolink:in_clinical_trials_for,PUBCHEM.COMPOUND:9831643
3,DXP04H,Non-hodgkin lymphoma,Approved,biolink:treats,PUBCHEM.COMPOUND:129269915
4,DXP04H,Chronic lymphocytic leukaemia,Phase 3,biolink:in_clinical_trials_for,PUBCHEM.COMPOUND:129269915
...,...,...,...,...,...
10185,D00EIL,Solid tumour/cancer,Investigative,biolink:studied_to_treat,PUBCHEM.COMPOUND:6419747
10186,D00DJC,Solid tumour/cancer,Investigative,biolink:studied_to_treat,PUBCHEM.COMPOUND:11488447
10187,D00DHM,Psychotic disorder,Investigative,biolink:studied_to_treat,PUBCHEM.COMPOUND:14277536
10188,D00DDR,Multiple myeloma,Investigative,biolink:studied_to_treat,PUBCHEM.COMPOUND:91865905


### Filter out some indication names

First, filter out some indication names that are known to be problematic - not "conditions that are treated" or I'm worried how the statement will look 

In [312]:
## EDA - used to help find problematic indications

test = "feline"

## set case=False so it isn't case-sensitive on matches!
df[df["object_indication_name"].str.contains(test, case=False)].shape

sorted(df[df["object_indication_name"].str.contains(test, case=False)]["object_indication_name"].unique())

(1, 5)

['Canine and feline spontaneous neoplasm']

In [313]:
## INCLUDE in parser

STRINGS_TO_FILTER = [
    "imaging",
    "radio",         ## related to imaging
    "esthesia",      ## for multiple spellings of an(a)esthesia
    "abortion",      ## problematic? but "spontaneous abortion" aka miscarriage can be treated...
    "sedation",
    "Discover", 
    "icide",         ## catches Herbicide, Insecticide, etc. But catches "poisoning" due to these things too
    "procedure",
    "barrier",       ## catches Blood brain barrier
    "astringent",
    "stimul",        ## catches "Caerulein stimulated..." and ovarian stimulation
    "suppress",      ## catches Appetite suppressant
    "contrast",      ## related to imaging
    "Diagnostic",    ## diagnostic
    "vasodilator",
    "Dutch elm disease",   ## this is a plant disease
    "Exam",
    "lubricant",
    "Localisation",
    "Measur",        ## catches Measure kidney function
    "Pest attack",   ## plant disease?
    "Plant grey",    ## catches Plant grey mould disease
    "Stabil",        ## catches Stabilize muscle contraction
    "canine",        ## Canine and feline spontaneous neoplasm
]

In [314]:
## INCLUDE in parser

## set case=False so it isn't case-sensitive on matches!
df = df[~df.object_indication_name.str.contains('|'.join(STRINGS_TO_FILTER), case=False)].copy()

## -> log
print(f"{df.shape[0]} after filtering out problematic indication names")

10054 after filtering out problematic indication names


In [315]:
## check that filtering worked

for i in STRINGS_TO_FILTER:
    if df[df["object_indication_name"].str.contains(i, case=False)].shape[0] > 0:
        print(i)

### EDA map indication names

Uses NameRes.

Using a score threshold to remove "low-quality" mappings, but there's a tradeoff...
* some "incorrect" mappings slip through - have high enough scores (300 < x < 500)
* some correct mappings get filtered out - have low scores (<300)

Decided to pick 300 as "good enough", but it has both problems

In [316]:
## get set of unique names to put into NameRes
indication_names = df["object_indication_name"].unique()

len(indication_names)

1222

In [50]:
## EDA version: for testing param settings

indication_mapping = {}
threshold = 300
batch_size = 500

## set up variables to catch mapping failures
stats_indication_mapping_failures = {
    "unexpected_error": {},
    "returned_empty": [],
    "score_under_threshold": {},
}
## for debug: stopping early
# counter = 0

for batch in batched(indication_names, batch_size):
    ## returns tuples -> cast to list
    req_body = {
        "strings": list(batch),
        "autocomplete": False,
        "limit": 1,  ## only analyzing top hit
        "biolink_types": ["DiseaseOrPhenotypicFeature"],
        "exclude_prefixes": "UMLS|MESH",  ## try finding high-quality hits
    }
    r = requests.post(NAMERES_URL, json=req_body)
    response = r.json()
        
    ## not doing dict comprehension. allows easier review, logic writing
    for k,v in response.items():
        ## catch unexpected errors
        try:
            ## will catch if v is an empty list (aka NameRes didn't have info)
            if v:
                ## v is a 1-element list, work with it directly
                temp = v[0]
                ## also throw out mapping if score < threshold: want better-matching hits
                if temp["score"] > threshold:
                    indication_mapping.update({
                        k: {"id": temp["curie"],
                            "label": temp["label"],
                           }
                    })
                else:
                    temp_data = {
                        "curie": temp["curie"],
                        "label": temp["label"],
                        "types": temp["types"][0],
                        "score": temp["score"],
                    }
                    stats_indication_mapping_failures["score_under_threshold"].update({k: temp_data})
            else:
                stats_indication_mapping_failures["returned_empty"].append(k)
        except Exception as e:
            stats_indication_mapping_failures["unexpected_error"].update({k: e})
            print(f"Encountered an unexpected error: {e}.")
            print(f"NameRes response key: {k}")
            print(f"NameRes response value: {v}")
    
#     counter += batch_size
#     if counter == 500:
#         break

Testing batch-size:
* 100: 57s
* 200: 56s
* **500: 49s**

In [51]:
len(indication_mapping)

stats_indication_mapping_failures.keys()

1051

dict_keys(['unexpected_error', 'returned_empty', 'score_under_threshold'])

In [52]:
stats_indication_mapping_failures["unexpected_error"]

len(stats_indication_mapping_failures["returned_empty"])
stats_indication_mapping_failures["returned_empty"]

{}

7

['Sterilant',
 'Contraception',
 'Kinetoplastids',
 'Vulnerary',
 'Chemoprotection',
 'Sweetener',
 'Expectorant']

In [53]:
len(stats_indication_mapping_failures["score_under_threshold"])

# ## to review 
# # sorted(stats_indication_mapping_failures["score_under_threshold"].keys())

164

In [67]:
## looking at KEYS for unexpected, missed delimiters

sorted(indication_mapping.keys())[1000:]

['Urticaria',
 'Uterine fibroids',
 'Uveitis',
 'Varicella zoster virus infection',
 'Varicose and spider veins of leg',
 'Varicose veins',
 'Vascular dementia',
 'Vascular disease',
 'Vascular purpura',
 'Vasomotor/allergic rhinitis',
 'Venous leg ulcer',
 'Venous thromboembolism',
 'Venous thrombosis',
 'Ventilator-associated pneumonia',
 'Ventricular arrhythmias',
 'Ventricular fibrillation',
 'Ventricular tachycardia',
 'Verruca vulgaris',
 'Vertigo meniere disease',
 'Vibrio cholerae infection',
 'Virus infection',
 'Visceral leishmaniasis',
 'Vitamin B1 deficiency',
 'Vitamin B12 deficiency',
 'Vitamin B6 deficiency',
 'Vitamin C deficiency',
 'Vitamin D deficiency',
 'Vitamin E deficiency',
 'Vitamin K deficiency',
 'Vitamin deficiency',
 'Vitiligo',
 'Vomiting',
 'Von hippel-lindau disease',
 'Vulvar intraepithelial neoplasia',
 'Vulvovaginal Candidiasis',
 'Vulvovaginitis',
 'Waldenstrom macroglobulinemia',
 'Weight loss',
 'Wernicke-Korsakoff Syndrome',
 'Wet age-related macu

### Parser map indication names

In [317]:
## INCLUDE in parser

## get set of unique names to put into NameRes
indication_names = df["object_indication_name"].unique()

## EDA only
len(indication_names)

1222

In [318]:
## function from parser

def run_nameres(names: Iterable[str], 
                url: str, 
                batch_size: int = 500, 
                types: list | None = None, 
                exclude_namespaces: str | None = None,
                score_threshold: int = 0,):
    """
    Parameters:
    - names: string names to NameRes (iterable - can be a set)
    - url: NameRes url/endpoint to use
    - batch_size: number of strings to include in 1 query to NameRes
    - types (default None): list of biolink categories that NameRes hits should have (hierarchy expansion is supported)
    - exclude_namespaces (default None): |-delimited string of ID namespaces to exclude, for quality
    - score_threshold (default 0): only accept hit if its score is greater than this, for quality

    Returns: tuple of mapping dict and failure stats dict
    """
    ## set up variables to collect output
    mapping = {}
    stats_failures = {
        "unexpected_error": {},
        "returned_empty": [],
        "score_under_threshold": [],
    }
#     ## for debug: stopping early
#     counter = 0

    for batch in batched(names, batch_size):
        req_body = {
            "strings": list(batch),  ## returns tuples -> cast to list
            "autocomplete": False,   ## names are complete search term
            "limit": 1,              ## only want to review top hit
            "biolink_types": types,
            "exclude_prefixes": exclude_namespaces,    ## try to increase quality of hits
        }    
        r = requests.post(url, json=req_body)
        response = r.json()

        ## not doing dict comprehension. allows easier review, logic writing
        for k,v in response.items():
            ## catch unexpected errors
            try:
                ## will catch if v is an empty list (aka NameRes didn't have info)
                if v:
                    ## v is a 1-element list, work with it directly
                    temp = v[0]
                    ## also throw out mapping if score < score_threshold: want better-matching hits
                    if temp["score"] > score_threshold:
                        mapping.update({
                            k: temp["curie"]
                        })
                    else:
                        stats_failures["score_under_threshold"].append(k)
                else:
                    stats_failures["returned_empty"].append(k)
            except Exception as e:
                stats_failures["unexpected_error"].update({k: e})

#         counter += batch_size
#         if counter >= 500:
#             break

    return mapping, stats_failures

In [321]:
## INCLUDE in parser

## param values
## use NAMERES_URL initialized earlier
# indication_batch_size = 500
indication_types = ["DiseaseOrPhenotypicFeature"]
indication_exclude_prefixes = "UMLS|MESH"
indication_score_threshold = 300


indication_mapping, stats_indication_mapping_failures = \
    run_nameres(names=indication_names,
                url=NAMERES_URL, 
                batch_size=indication_batch_size,
                types=indication_types,
                exclude_namespaces=indication_exclude_prefixes,
                score_threshold=indication_score_threshold,
    )

In [322]:
## EDA only

len(indication_mapping)

stats_indication_mapping_failures.keys()

stats_indication_mapping_failures["unexpected_error"]
stats_indication_mapping_failures["returned_empty"]

len(stats_indication_mapping_failures["score_under_threshold"])

1051

dict_keys(['unexpected_error', 'returned_empty', 'score_under_threshold'])

{}

['Sterilant',
 'Contraception',
 'Kinetoplastids',
 'Vulnerary',
 'Chemoprotection',
 'Sweetener',
 'Expectorant']

164

In [323]:
## INCLUDE in parser

## get method returns None if key (indication name) not found in mapping 
df["object_nameres_id"] = [indication_mapping.get(i) for i in df["object_indication_name"]]

In [324]:
## INCLUDE in parser -> log

n_mapped = df["object_nameres_id"].notna().sum()
print(f"{n_mapped} rows with mapped indication names: {n_mapped / df.shape[0]:.1%}")

8350 rows with mapped indication names: 83.1%


**~83% of the data could be mapped.**

In [325]:
## INCLUDE in parser

## drop rows without indication mapping
df.dropna(subset="object_nameres_id", inplace=True, ignore_index=True)

In [326]:
df.shape

(8350, 6)

### Merge "duplicates" after mapping

<div class="alert alert-block alert-danger">

[2025-11-03]

With the current pipeline and data-modeling, **only the mapped columns uniquely define an edge**:
* subject_pubchem
* biolink_predicate
* object_nameres_id

**We still care about...**
* `subject_ttd_drug`: used to generate links to TTD in `source_record_urls`. That property can be multi-valued (handle mapping to multiple original TTD IDs)

And the rest won't be included in the output edges:
* `object_indication_name`: would be nice to include as "original_object" edge property, but currently the pipeline will override it with the provided object ID (object_nameres_id). Also it shouldn't be multi-valued right now
* `clinical_status`: not including in output right now. Needs to be mapped to edge-attribute(s) and values - then those properties could be included in "unique definition of an edge". 

In [327]:
## INCLUDE in parser

cols_define_edge = ["subject_pubchem", "biolink_predicate", "object_nameres_id"]
df = df.groupby(by=cols_define_edge).agg(set).reset_index().copy()

In [328]:
## EDA only

df[df["subject_ttd_drug"].map(len) > 1].shape

df[df["object_indication_name"].map(len) > 1].shape

df[df["clinical_status"].map(len) > 1].shape

(51, 6)

(25, 6)

(7, 6)

In [329]:
## EDA only

df[df["clinical_status"].map(len) > 1]

,subject_pubchem,biolink_predicate,object_nameres_id,subject_ttd_drug,object_indication_name,clinical_status
1624,PUBCHEM.COMPOUND:136033680,biolink:in_clinical_trials_for,MONDO:0018177,{D0V8AG},"{Glioblastoma multiforme, Recurrent glioblastoma}","{Phase 2, Phase 1/2}"
1925,PUBCHEM.COMPOUND:15950826,biolink:in_clinical_trials_for,MONDO:0056806,{D0X8TS},"{Non-small-cell lung cancer, Small-cell lung cancer}","{Phase 3, Phase 2}"
3111,PUBCHEM.COMPOUND:25195461,biolink:in_clinical_trials_for,MONDO:0005027,"{D0TS8E, D2JF0S}",{Epilepsy},"{Phase 1, Phase 2}"
3453,PUBCHEM.COMPOUND:3083544,biolink:treats,MONDO:0004979,"{D0XW2H, D0D1DI}",{Asthma},"{Approved, Phase 4}"
6101,PUBCHEM.COMPOUND:60825,biolink:in_clinical_trials_for,MONDO:0022839,"{D01SKR, D02UVT}",{Human immunodeficiency virus infection},"{Phase 3, Phase 2}"
7412,PUBCHEM.COMPOUND:85364156,biolink:in_clinical_trials_for,MONDO:0006769,{D0U2CE},"{Gastroparesis, Diabetic gastroparesis}","{Phase 3, Phase 2}"
7552,PUBCHEM.COMPOUND:89683805,biolink:in_clinical_trials_for,MONDO:0018874,{D0K7FT},"{Acute myeloid leukaemia, Acute myelogenous leukaemia}","{Phase 3, Phase 1/2}"


In [330]:
## INCLUDE in parser -> log

print(f"{df.shape[0]} rows after handling 'edge-level' duplicates\n")

print(f"{df["subject_pubchem"].nunique()} unique mapped drug IDs")

print(f"{df["object_nameres_id"].nunique()} unique mapped DiseaseOrPheno IDs")

# print(f"{df["clinical_status"].nunique()} unique clinical status values")

8276 rows after handling 'edge-level' duplicates

6048 unique mapped drug IDs
907 unique mapped DiseaseOrPheno IDs


In [ ]:
## pipeline output would use to_dict
# df.to_dict(orient="records")

### EDA generating TTD links

In [54]:
## writing out here because it's two loops 

for i in df["subject_ttd_drug"]:
    if len(list(i)) > 1:
        ## INCLUDE IN PARSER
        temp = ["https://ttd.idrblab.cn/data/drug/details/" + j.lower() for j in i]
        print(temp)
        break

['https://ttd.idrblab.cn/data/drug/details/d02eyg', 'https://ttd.idrblab.cn/data/drug/details/d05jac']


## P1-07: "drug affects protein"

### Load file

Simpler because this is a regular tabular xlsx Excel file

In [142]:
p1_07_path = base_file_path.joinpath("P1-07-Drug-TargetMapping.xlsx")

In [248]:
# requires openpyxl to run
## na_values comes from manual EDA of excel file
df_07 = pd.read_excel(io=p1_07_path, na_values=".")

## INCLUDE IN PARSER: don't include Highest_status column
# df_07 = pd.read_excel(io=p1_07_path, usecols=["TargetID", "DrugID", "MOA"])

### EDA

In [144]:
df_07.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45453 entries, 0 to 45452
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   TargetID        45453 non-null  object
 1   DrugID          45453 non-null  object
 2   Highest_status  45453 non-null  object
 3   MOA             43200 non-null  object
dtypes: object(4)
memory usage: 1.4+ MB


In [145]:
## number of edge IDs vs mapping

df_07["TargetID"].nunique()
len(ttd_target_mappings)

len(set(df_07["TargetID"].unique()) - set(ttd_target_mappings.keys()))

3197

3629

534

In [146]:
## look at it
list(set(df_07["TargetID"].unique()) - set(ttd_target_mappings.keys()))[0:10]

['T04389',
 'T39554',
 'T10339',
 'T76563',
 'T97553',
 'T12005',
 'T45598',
 'T83717',
 'T07861',
 'T24835']

In [147]:
x = 'T04389'

ttd_target_mappings.get(x)

df_07[df_07["TargetID"] == x]

,TargetID,DrugID,Highest_status,MOA
25932,T04389,D0IX7G,Terminated,Modulator
27918,T04389,D09PGM,Discontinued in Phase 2,Modulator
35007,T04389,D07KIE,Phase 2,Modulator


There are more mappings than unique target IDs in the edges. 

But there's still a good chunk of IDs in the data that aren't covered by the mappings. 
And the edge data doesn't have any names that I can use to get IDs using NameRes.  

In [148]:
## number of edge IDs vs mapping

df_07["DrugID"].nunique()
len(ttd_drug_mappings)

32824

25423

There are less mappings than unique drug IDs in the edges. 

So just by numbers, there'll be a lot of edge data lost because there aren't mapped IDs. 

In [149]:
df_07["Highest_status"].nunique()

df_07["Highest_status"].value_counts()

45

Highest_status
Investigative                      24870
Patented                            6017
Phase 2                             2838
Phase 1                             2805
Approved                            2735
Terminated                          1448
Phase 3                             1118
Discontinued in Phase 2              922
Phase 1/2                            810
Discontinued in Phase 1              576
Preclinical                          529
Discontinued in Phase 3              219
Phase 2/3                            138
Clinical trial                        74
Withdrawn from market                 56
Phase 4                               54
Phase 3 Trial                         48
Discontinued in Phase 1/2             33
Phase 2 Trial                         27
Discontinued in Preregistration       26
Preregistration                       12
Phase 2a                              11
Phase 2/3 Trial                       11
Phase 1 Trial                          9
I

This looks very similar to the clinical_status from P1-05. However, it's not clear to me if this status is for the target or the drug (or basically both)? 

### MOA parsing

In [249]:
## EDA

df_07["MOA"].nunique(dropna=False)

sorted(df_07["MOA"].dropna().unique().tolist())

55

['Activator',
 'Agonis; Inverse agonist',
 'Agonist',
 'Agonist ',
 'Antagonist',
 'Antagonist ',
 'Antagonist/GLP1 agonist',
 'Antisense ',
 'Binder',
 'Binder (minor groove binder)',
 'Blocker',
 'Blocker (channel blocker)',
 'Breaker',
 'CAR-T-Cell-Therapy',
 'CAR-T-Cell-Therapy(Dual specific)',
 'Chelator',
 'Co-agonist',
 'Cofactor',
 'Degrader',
 'Degrader ',
 'Disrupter',
 'Enhancer',
 'Immune response agent',
 'Immunomodulator',
 'Immunomodulator ',
 'Immunomodulator (Immunostimulant)',
 'Immunostimulant',
 'Inactivator',
 'Inducer',
 'Inhibitor',
 'Inhibitor (gating inhibitor)',
 'Inhibitor; Antagonist; Blocker',
 'Intercalator',
 'Inverse agonist',
 'Ligand',
 'Mimetic',
 'Modulator',
 'Modulator ',
 'Modulator (allosteric modulator)',
 'Opener',
 'Partial agonist',
 'Potentiator',
 'Reactivator',
 'Regulator',
 'Regulator (upregulator)',
 'Replacement',
 'Silencer',
 'Stabilizer',
 'Stablizer',
 'Stimulator',
 'Stimulator ',
 'Suppressor',
 'antagonist',
 'inhibitor']

In [250]:
## INCLUDE in parser

## split on ";"

df_07["MOA"] = df_07["MOA"].str.split(";")

In [251]:
## INCLUDE in parser

df_07.shape[0]

## expand to multiple rows when MOA list length > 1
## also pops every MOA value out into a string
df_07 = df_07.explode("MOA", ignore_index=True)

## -> log
print(f"{df_07.shape[0]} rows after expanding MOAs with multiple values")

45453

45468 rows after expanding mappings with multiple ID values


In [252]:
## INCLUDE in parser

## clean up things that aren't actually unique values

## remove whitespace 
df_07["MOA"] = df_07["MOA"].str.strip()

## make case consistent - have stuff like "Inhibitor" vs "inhibitor"
df_07["MOA"] = df_07["MOA"].str.lower()

In [253]:
## INCLUDE in parser


## typos

## EDA only
df_07[df_07["MOA"].str.contains("agonis$", na=False, regex=True)]
df_07[df_07["MOA"] == "stablizer"]  ## should be stabilizer

## replacements are trickier because there's NA values in column
## replaces np.nan with None
df_07["MOA"] = [re.sub("agonis$", "agonist", i) if pd.notna(i) else None for i in df_07["MOA"]]

df_07["MOA"] = [re.sub("stablizer", "stabilizer", i) if pd.notna(i) else None for i in df_07["MOA"]]

,TargetID,DrugID,Highest_status,MOA
29532,T64765,D0B2NV,Patented,agonis


,TargetID,DrugID,Highest_status,MOA
29698,T20761,D0WL5P,Patented,stablizer
33279,T43189,D04NXL,Phase 2,stablizer
35883,T43189,D07WPE,Phase 3,stablizer
37286,T43189,D0W2EK,Approved,stablizer


In [254]:
## EDA 

df_07[df_07["MOA"].str.contains("agonis$", na=False, regex=True)]

df_07[(df_07["TargetID"] == "T64765") & (df_07["DrugID"] == "D0B2NV")]

,TargetID,DrugID,Highest_status,MOA


,TargetID,DrugID,Highest_status,MOA
29532,T64765,D0B2NV,Patented,agonist
29533,T64765,D0B2NV,Patented,inverse agonist


In [255]:
## EDA 

df_07[df_07["MOA"].str.contains("stablizer", na=False)]

df_07[(df_07["TargetID"] == "T43189") & (df_07["DrugID"] == "D0W2EK")]

,TargetID,DrugID,Highest_status,MOA


,TargetID,DrugID,Highest_status,MOA
37286,T43189,D0W2EK,Approved,stabilizer


In [256]:
## EDA: after

df_07["MOA"].nunique(dropna=False)

df_07["MOA"].value_counts(dropna=False)

44

MOA
inhibitor                            29748
modulator                             4737
antagonist                            3569
agonist                               2691
None                                  2253
activator                              389
binder                                 374
car-t-cell-therapy                     349
modulator (allosteric modulator)       260
blocker                                244
ligand                                 208
blocker (channel blocker)              205
car-t-cell-therapy(dual specific)       78
replacement                             63
stimulator                              58
opener                                  39
inhibitor (gating inhibitor)            29
degrader                                20
inducer                                 20
immunomodulator                         19
enhancer                                17
breaker                                 14
reactivator                             10
suppres

In [257]:
## INCLUDE IN PARSER
cleaned_moa = set(df_07["MOA"].dropna().unique().tolist())
cleaned_moa

{'activator',
 'agonist',
 'antagonist',
 'antagonist/glp1 agonist',
 'antisense',
 'binder',
 'binder (minor groove binder)',
 'blocker',
 'blocker (channel blocker)',
 'breaker',
 'car-t-cell-therapy',
 'car-t-cell-therapy(dual specific)',
 'chelator',
 'co-agonist',
 'cofactor',
 'degrader',
 'disrupter',
 'enhancer',
 'immune response agent',
 'immunomodulator',
 'immunomodulator (immunostimulant)',
 'immunostimulant',
 'inactivator',
 'inducer',
 'inhibitor',
 'inhibitor (gating inhibitor)',
 'intercalator',
 'inverse agonist',
 'ligand',
 'mimetic',
 'modulator',
 'modulator (allosteric modulator)',
 'opener',
 'partial agonist',
 'potentiator',
 'reactivator',
 'regulator',
 'regulator (upregulator)',
 'replacement',
 'silencer',
 'stabilizer',
 'stimulator',
 'suppressor'}

Notes on some with no data-modeling mapping:
* 'antagonist/glp1 agonist': dunno if this should be split into 2 diff mechanisms, or treated together (GLP1 agonist is special kind of antagonism?)
* 'binder (minor groove binder)': sounds like a special kind of binding to DNA, not a regular "binder" 
* 'immunomodulator (immunostimulant)': dunno if this should be split into 2 diff mechanisms, or treated together (same as plain 'immunostimulant'?)

Unsure:
* diff between 'blocker' and 'blocker (channel blocker)': gave same modeling to both (molecular_channel_blockage)

### Map TTD drug IDs

Uses `ttd_drug_mappings` generated earlier when parsing mapping files (P1-03). 

If mappings had more namespaces (logic needed to pick one), would write it out - not use list comprehension:
* If getting TTD ID successful...
  * If getting pubchem successful...use it
  * Else, use inchikey
 * Else: use None

In [258]:
## INCLUDE in parser

## get method returns None if key (TTD ID) not found in mapping 
df_07["subject_pubchem"] = [ttd_drug_mappings.get(i) for i in df_07["DrugID"]]

In [259]:
## EDA - check that no value is "PUBCHEM.COMPOUND:"

df_07[df_07["subject_pubchem"] == "PUBCHEM.COMPOUND:"]

,TargetID,DrugID,Highest_status,MOA,subject_pubchem


In [260]:
## INCLUDE in parser -> log

n_mapped = df_07["subject_pubchem"].notna().sum()
print(f"{n_mapped} rows with mapped TTD drug IDs: {n_mapped / df_07.shape[0]:.1%}")

34166 rows with mapped TTD drug IDs: 75.1%


**~75% of the data could be mapped.**

In [261]:
## INCLUDE in parser

## remove rows without drug mapping
df_07.dropna(subset="subject_pubchem", inplace=True, ignore_index=True)

In [262]:
## EDA only

df_07.shape

(34166, 5)

In [263]:
## EDA: multiple pubchem values? 

df_07[df_07["subject_pubchem"].map(len) > 1].shape

df_07[df_07["subject_pubchem"].map(len) > 1]

(18, 5)

,TargetID,DrugID,Highest_status,MOA,subject_pubchem
6531,T11253,D00ZBZ,Investigative,None,"[PUBCHEM.COMPOUND:91866823, PUBCHEM.COMPOUND:36314]"
6533,T47387,D0S1FM,Investigative,None,"[PUBCHEM.COMPOUND:91865905, PUBCHEM.COMPOUND:36314]"
6534,T47387,D0LM3T,Investigative,None,"[PUBCHEM.COMPOUND:91865905, PUBCHEM.COMPOUND:657237]"
6535,T47387,D0XU9F,Investigative,None,"[PUBCHEM.COMPOUND:91865905, PUBCHEM.COMPOUND:6253]"
6565,T47387,D0XU1I,Investigative,None,"[PUBCHEM.COMPOUND:91865905, PUBCHEM.COMPOUND:5353562]"
6566,T47387,D00DDR,Investigative,None,"[PUBCHEM.COMPOUND:91865905, PUBCHEM.COMPOUND:5426]"
6567,T47387,D0W8ZE,Investigative,None,"[PUBCHEM.COMPOUND:91865905, PUBCHEM.COMPOUND:5743]"
6568,T47387,D05IAN,Investigative,None,"[PUBCHEM.COMPOUND:91865905, PUBCHEM.COMPOUND:60838]"
6571,T06413,D0K4GI,Investigative,None,"[PUBCHEM.COMPOUND:71587700, PUBCHEM.COMPOUND:60750]"
6572,T06413,D02MFK,Investigative,None,"[PUBCHEM.COMPOUND:71587700, PUBCHEM.COMPOUND:84093]"


In [264]:
## INCLUDE in parser

## expand to multiple rows when subject_pubchem list length > 1
## also pops every subject_pubchem value out into a string
df_07 = df_07.explode("subject_pubchem", ignore_index=True)

## -> log
print(f"{df_07.shape[0]} rows after expanding mappings with multiple ID values")

## EDA only
# df_07

34184 rows after expanding mappings with multiple ID values


### Map TTD target IDs

Uses `ttd_target_mappings` generated earlier when parsing mapping files (P2-01)

In [266]:
## INCLUDE in parser

## get method returns None if key (TTD ID) not found in mapping 
df_07["object_id"] = [ttd_target_mappings[i]["mapped_ids"] if ttd_target_mappings.get(i) else None for i in df_07["TargetID"]]

In [267]:
## INCLUDE in parser -> log

n_mapped = df_07["object_id"].notna().sum()
print(f"{n_mapped} rows with mapped TTD target IDs: {n_mapped / df_07.shape[0]:.1%}")

31289 rows with mapped TTD drug IDs: 91.5%


**~91% of the data could be mapped.**

In [268]:
## INCLUDE in parser

## remove rows without target mapping
df_07.dropna(subset="object_id", inplace=True, ignore_index=True)

In [269]:
## EDA

df_07.shape

(31289, 6)

In [270]:
## EDA: multiple pubchem values? 

df_07[df_07["object_id"].map(len) > 1].shape

df_07[df_07["object_id"].map(len) > 1]

(809, 6)

,TargetID,DrugID,Highest_status,MOA,subject_pubchem,object_id
20,T38284,DSHT18,Approved,inhibitor,PUBCHEM.COMPOUND:72165228,"[NCBIGene:25, NCBIGene:613]"
137,T90286,D0VF6E,Investigative,inhibitor,PUBCHEM.COMPOUND:1401,"[NCBIGene:2261, NCBIGene:10460]"
143,T55285,D04ZCZ,Terminated,inhibitor,PUBCHEM.COMPOUND:65243,"[NCBIGene:2741, NCBIGene:8001, NCBIGene:2743, NCBIGene:2742]"
187,T49639,D09VWI,Phase 1,modulator,PUBCHEM.COMPOUND:6421776,"[UniProtKB:P0DMV8, UniProtKB:P0DMV9]"
221,T22348,D4M8RS,Phase 2,inhibitor,PUBCHEM.COMPOUND:145722022,"[UniProtKB:P0A0K8, UniProtKB:P20831]"
...,...,...,...,...,...,...
31174,T85042,DMBQ10,Phase 1/2,agonist,PUBCHEM.COMPOUND:21158511,"[NCBIGene:5467, NCBIGene:5468, NCBIGene:5465]"
31202,T57680,D2RG4D,Phase 2,activator,PUBCHEM.COMPOUND:134440946,"[NCBIGene:2982, NCBIGene:2983]"
31204,T63206,D3MG8W,Phase 2,inhibitor,PUBCHEM.COMPOUND:135390719,"[NCBIGene:3688, NCBIGene:3685]"
31205,T71141,D3MG8W,Phase 2,inhibitor,PUBCHEM.COMPOUND:135390719,"[NCBIGene:3694, NCBIGene:3685]"


In [271]:
## INCLUDE in parser

## expand to multiple rows when object_id list length > 1
df_07 = df_07.explode("object_id", ignore_index=True)

## -> log
print(f"{df_07.shape[0]} rows after expanding mappings with multiple ID values")

## EDA only
# df_07

32642 rows after expanding mappings with multiple ID values


### Merge "duplicates" after mapping

<div class="alert alert-block alert-danger">

[2025-12-07]

With the current pipeline and data-modeling, **only the mapped columns uniquely define an edge**:
* subject_pubchem
* object_id
* unique data-modeling based on MOA

**We still care about...**
* `subject_ttd_drug`: used to generate links to TTD in `source_record_urls`. That property can be multi-valued (handle mapping to multiple original TTD IDs). [Using target page will sometimes not work](https://github.com/NCATSTranslator/Data-Ingest-Coordination-Working-Group/issues/30#issuecomment-3209944640)

And the rest won't be included in the output edges:
* `Highest_status`

First make a new column with unique value for each data-modeling

In [275]:
## current MOA values with the same data-modeling

BINDING_TYPES = {"binder", "ligand"}

BLOCKING_TYPES = {"blocker", "blocker (channel blocker)"}

In [276]:
df_07["mod_moa"] = [
    "BINDING" if i in BINDING_TYPES \
    else "BLOCKING" if i in BLOCKING_TYPES \
    else i
    for i in df_07["MOA"]
]

In [277]:
## EDA, seeing if it worked

set(df_07["mod_moa"].unique().tolist())

{'BINDING',
 'BLOCKING',
 None,
 'activator',
 'agonist',
 'antagonist',
 'antisense',
 'breaker',
 'car-t-cell-therapy',
 'cofactor',
 'degrader',
 'disrupter',
 'enhancer',
 'immunomodulator',
 'inducer',
 'inhibitor',
 'inhibitor (gating inhibitor)',
 'intercalator',
 'inverse agonist',
 'mimetic',
 'modulator',
 'modulator (allosteric modulator)',
 'opener',
 'potentiator',
 'regulator',
 'regulator (upregulator)',
 'replacement',
 'stimulator',
 'suppressor'}

In [279]:
## only for notebook: remove Highest_status column

df_07.columns

df_07.drop(columns="Highest_status", inplace=True)

df_07.columns

Index(['TargetID', 'DrugID', 'Highest_status', 'MOA', 'subject_pubchem',
       'object_id', 'mod_moa'],
      dtype='object')

Index(['TargetID', 'DrugID', 'MOA', 'subject_pubchem', 'object_id', 'mod_moa'], dtype='object')

In [281]:
## INCLUDE in parser

cols_define_edge = ["subject_pubchem", "object_id", "mod_moa"]
df_07 = df_07.groupby(by=cols_define_edge).agg(set).reset_index().copy()

In [285]:
## EDA only

df_07[df_07["TargetID"].map(len) > 1].shape
df_07[df_07["TargetID"].map(len) > 1][0:5]


df_07[df_07["DrugID"].map(len) > 1].shape
df_07[df_07["DrugID"].map(len) > 1][0:5]


df_07[df_07["MOA"].map(len) > 1]

(112, 6)

,subject_pubchem,object_id,mod_moa,TargetID,DrugID,MOA
10,PUBCHEM.COMPOUND:10005132,UniProtKB:P49356,inhibitor,"{T13127, T86428}",{D0R3PD},{inhibitor}
338,PUBCHEM.COMPOUND:10102380,UniProtKB:P49356,inhibitor,"{T13127, T86428}",{D01OFD},{inhibitor}
806,PUBCHEM.COMPOUND:102593999,NCBIGene:2146,inhibitor,"{T04980, T25265}",{D0D4UH},{inhibitor}
1345,PUBCHEM.COMPOUND:10432334,UniProtKB:P49356,inhibitor,"{T86428, T13127}",{D0L5AS},{inhibitor}
1414,PUBCHEM.COMPOUND:10455969,NCBIGene:5580,inhibitor,"{T44861, T49368}",{D06ZCX},{inhibitor}


(39, 6)

,subject_pubchem,object_id,mod_moa,TargetID,DrugID,MOA
2384,PUBCHEM.COMPOUND:11110698,NCBIGene:4907,inhibitor,{T65116},"{D0MA9U, D0Y2QO}",{inhibitor}
2973,PUBCHEM.COMPOUND:11374217,NCBIGene:387129,antagonist,{T20958},"{D03EJP, D06FJL}",{antagonist}
3042,PUBCHEM.COMPOUND:11396600,NCBIGene:7421,agonist,{T34234},"{D0I4GI, D00SZR}",{agonist}
3979,PUBCHEM.COMPOUND:11617559,NCBIGene:1436,inhibitor,"{T55414, T20333}","{D0X9VS, D0D6MH}",{inhibitor}
4578,PUBCHEM.COMPOUND:117807842,NCBIGene:2146,inhibitor,{T25265},"{D09PQQ, D08MXO}",{inhibitor}


,subject_pubchem,object_id,mod_moa,TargetID,DrugID,MOA
6627,PUBCHEM.COMPOUND:1345,NCBIGene:706,BINDING,{T75440},"{D0VN4K, D01KPW}","{ligand, binder}"
9698,PUBCHEM.COMPOUND:1688,NCBIGene:706,BINDING,{T75440},"{D0B1PE, D0R2XX}","{ligand, binder}"


In [286]:
## INCLUDE in parser -> log

print(f"{df_07.shape[0]} rows after handling 'edge-level' duplicates\n")

print(f"{df_07["subject_pubchem"].nunique()} unique mapped drug IDs")

print(f"{df_07["object_id"].nunique()} unique mapped target IDs")

## doesn't work because MOA is now a set

32367 rows after handling 'edge-level' duplicates

21026 unique mapped drug IDs
1941 unique mapped target IDs


In [ ]:
## pipeline output would use to_dict
# df.to_dict(orient="records")

### EDA generating TTD links

In [287]:
## writing out here because it's two loops 

for i in df_07["TargetID"]:
    if len(list(i)) > 1:
        ## INCLUDE IN PARSER
        temp = ["https://ttd.idrblab.cn/data/drug/details/" + j.lower() for j in i]
        print(temp)
        break

['https://ttd.idrblab.cn/data/drug/details/t13127', 'https://ttd.idrblab.cn/data/drug/details/t86428']


### For comparing to pipeline output

In [290]:
## count after adding extra edges

## add escape characters for ()
augmented = ["agonist", "antagonist",
             "BLOCKING", 
             "inhibitor", "inhibitor \\(gating inhibitor\\)",
             "inverse agonist", "partial agonist"
            ]

df_07.shape[0] + \
df_07[df_07.mod_moa.str.contains('|'.join(augmented), case=False)].shape[0]

61200